In [5]:
import csv
import tensor_calculation as t
import tensor_advanced as ta
from collections import Counter
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
import random
import operator
from math import exp
from __future__ import division

In [ ]:
#implement a simple twitter sentiment classifier

nltk.download('punkt')

nltk.download('stopwords')

In [6]:
import sys 
reload(sys)  
sys.setdefaultencoding('latin-1')

In [7]:
f = open("training.1600000.processed.noemoticon.csv")
ff = csv.reader(f)
data0=[]
for row in ff:
    data0.append(row)
data = [[d[0], d[5]] for d in data0]

In [8]:
pre_data = [word_tokenize(dd[1])  for dd in data[:20000]]
pre_data1 = [word_tokenize(dd[1])  for dd in data[-20000:]]

In [9]:
len(data)

1600000

tol = {}
for w0 in pre_data:
    w1 = [w_.lower() for w_ in w0 if w_.lower() not in stopwords.words('english')]
    w2 = [w_ for w_ in w1 if len(w_) > 1]
    w3 = [w_ for w_ in w2 if w_[0] not in ["/", "'", "+","." ]+["%r" %(i) for i in range(10)] ]
    w4 = [w_ for w_ in w2 if w_[-1:] not in ["%r" %(i) for i in range(10)] ]
    w5 = [w_ for w_ in w4if w_[:3] not in [ 'htt', 'www']]
    w6 = [w_ for w_ in w5 if "_" not in w_ ]
    for w  in w5:
        if tol.has_key(w):
            tol[w] +=1              
        else:
            tol[w] =1
finish=clock()
print finish-start

In [10]:
tol = {}
for twitte in pre_data:
    for word1 in twitte:
        word = word1.lower()
        if word  in stopwords.words('english')\
        or len(word) < 2\
        or word[0]  in ["/", "'", "+","." ]+["%r" %(i) for i in range(10)]\
        or word[-1:]  in ["%r" %(i) for i in range(10)]\
        or word[:3]  in [ 'htt', 'www']\
        or "_" in word:
            continue
        if tol.has_key(word):
            tol[word] +=1              
        else:
            tol[word] =1

In [11]:
tol1 = {}
for twitte in pre_data1:
    for word1 in twitte:
        word = word1.lower()
        if word  in stopwords.words('english')\
        or len(word) < 2\
        or word[0]  in ["/", "'", "+","." ]+["%r" %(i) for i in range(10)]\
        or word[-1:]  in ["%r" %(i) for i in range(10)]\
        or word[:3]  in [ 'htt', 'www']\
        or "_" in word:
            continue
        if tol1.has_key(word):
            tol1[word] +=1              
        else:
            tol1[word] =1

sorted(tol.items(), key=operator.itemgetter(1), reverse = True)

sorted(tol1.items(), key=operator.itemgetter(1), reverse = True)

In [12]:
whole_dict = {}   #the word  that appears most frequently  are most  useful
whole_dict.update(tol1)
for item in tol:
    if whole_dict.has_key(item):
        whole_dict[item] += tol[item]
    else:
        whole_dict[item] = tol[item]

In [13]:
diff_dict ={}      #the word whose counter has a big different between the two group will be a good indicator 
diff_dict.update(tol1)
for item in tol:
    if diff_dict.has_key(item):
        diff_dict[item] -= tol[item]
    else:
        diff_dict[item] = tol[item]

In [14]:
ranking_dict = {}    # using the two factor to decide the indicator words
for word in whole_dict:
    ranking_dict[word] = diff_dict[word]* whole_dict[word]**0.25

In [15]:
key_ = sorted(ranking_dict.items(), key=operator.itemgetter(1), reverse = True)[:500]+\
 sorted(ranking_dict.items(), key=operator.itemgetter(1))[:500]     #the ranking of frequent words in negative twitte 
key_words = [k[0] for k in key_]

In [16]:
def logist(list_):
    if type(list_) != list:
        return (1/(1+exp(-1*list_)))

In [17]:
def de_logist(list_):
    return t.list_muti(logist(list_), t.list_shift(t.list_nu(logist(list_), -1) , 1))

In [18]:
def twitte_to_x(twitte, key_words ):
    usefull = [ word.lower() for word in twitte  if  word.lower() in key_words]
    key_counter = Counter(usefull)
    return [key_counter[key]  for key in  key_words]

In [36]:
training_data=[]
for (i,j) in zip (pre_data1, pre_data):
    training_data.extend([[i , 1], [ j, 0] ])

In [50]:
def train(data):
    w= ta.gene_random_tensor([len(key_words)], (-1,1))
    while True:
        error = 0
        for d in data:
            x = twitte_to_x(d[0], key_words)
            wx = t.list_contrac(x, w)
            p = logist(wx)
            if (p-0.5)*(d[1]-0.5)<=0  :
                error +=1
            w = t.list_add(t.list_nu(x, -5*(p- d[1])* de_logist(wx)), w)
        #w = t.list_nu(w, 1/t.list_contrac(w, w))
        print error
        if error< 10000:
            break
    return w

In [51]:
train(training_data)

[6.629195524709935,
 12.47283509272293,
 7.746116782758679,
 5.738747975561715,
 5.815909824358254,
 0.5341827469531918,
 5.161936738026337,
 9.907357006964485,
 11.606417187020355,
 1.5698138769561916,
 15.482668679860753,
 3.2115915877767653,
 6.247521071832239,
 7.318699921434076,
 10.24162482224717,
 15.351699923356117,
 4.521232146005536,
 10.917326829171346,
 34.02094831802816,
 12.333089867882938,
 3.233926184293181,
 4.651123721158047,
 6.678905783762227,
 1.1007943374609461,
 5.187805416810236,
 20.946239190317684,
 1.6367842326944486,
 -1.6078968437271781,
 20.7343444792855,
 5.288976685435944,
 7.264506758512996,
 7.253144307528993,
 15.1084025054826,
 5.255738602883017,
 28.6294142771013,
 8.963540213139654,
 7.551466561150482,
 5.1674981540176566,
 4.511164556872963,
 0.8479203072773817,
 5.177885392709566,
 5.319776593632815,
 0.17351827142675774,
 4.507609542175656,
 11.158522083205451,
 3.721923843623569,
 2.5969654641602196,
 -0.9909856049158323,
 9.871446820187323,
 1

In [19]:
w=[6.629195524709935,
 12.47283509272293,
 7.746116782758679,
 5.738747975561715,
 5.815909824358254,
 0.5341827469531918,
 5.161936738026337,
 9.907357006964485,
 11.606417187020355,
 1.5698138769561916,
 15.482668679860753,
 3.2115915877767653,
 6.247521071832239,
 7.318699921434076,
 10.24162482224717,
 15.351699923356117,
 4.521232146005536,
 10.917326829171346,
 34.02094831802816,
 12.333089867882938,
 3.233926184293181,
 4.651123721158047,
 6.678905783762227,
 1.1007943374609461,
 5.187805416810236,
 20.946239190317684,
 1.6367842326944486,
 -1.6078968437271781,
 20.7343444792855,
 5.288976685435944,
 7.264506758512996,
 7.253144307528993,
 15.1084025054826,
 5.255738602883017,
 28.6294142771013,
 8.963540213139654,
 7.551466561150482,
 5.1674981540176566,
 4.511164556872963,
 0.8479203072773817,
 5.177885392709566,
 5.319776593632815,
 0.17351827142675774,
 4.507609542175656,
 11.158522083205451,
 3.721923843623569,
 2.5969654641602196,
 -0.9909856049158323,
 9.871446820187323,
 12.729935519588661,
 3.8732328398389395,
 8.175684280797165,
 11.786595932575013,
 10.653717010568295,
 2.8756087915785584,
 6.082896888270694,
 17.098861921296297,
 4.751601514880168,
 15.12592229500367,
 4.130999790340506,
 7.126329623103499,
 6.951676632096037,
 8.637548000115077,
 7.430067696668929,
 1.0946031777627092,
 1.602298672918408,
 14.275927975292744,
 1.7444113755482786,
 0.6323130252368402,
 10.608049119542738,
 -0.4843885109954008,
 3.4733974005433152,
 5.329729358738466,
 8.465431903650204,
 9.583190178922573,
 2.771556984242584,
 7.427538672928736,
 -0.5889956941230873,
 0.043011840603521805,
 2.5386128859839943,
 7.174354819864589,
 2.8585392743195714,
 5.02949867782799,
 8.634003118501663,
 1.1449198353436278,
 5.281306783556691,
 10.279410021169525,
 -0.159399755159459,
 9.426161232256543,
 5.13435259147375,
 2.8137177615804254,
 13.87566869635017,
 3.9734082690437815,
 2.1061013727803406,
 1.2709323280291562,
 9.203324402630768,
 7.648624148956416,
 4.572004550949613,
 22.086405975931896,
 1.1500601949741114,
 -0.7382159193673477,
 16.117206983939518,
 -0.592300520465852,
 7.686104896362642,
 16.77462003021375,
 1.611926313281185,
 5.356357238413424,
 32.10061337862111,
 9.812851686358593,
 14.323117167835319,
 4.9682180034693175,
 4.944381596083223,
 2.3988711336990263,
 5.3628682930516245,
 18.20703836477434,
 1.3088065254302585,
 12.378312065796957,
 8.40281575101727,
 3.7448468554596963,
 7.665795722554887,
 0.3903659084814248,
 0.42148886392671175,
 -0.6767794836366755,
 9.937564920229356,
 7.989920832135554,
 5.22272353730548,
 -1.8058513260240194,
 21.716871876980925,
 3.223964425363925,
 10.260917687236883,
 6.2407471399630605,
 17.17368904690856,
 20.191161758760146,
 3.545162978526136,
 2.9688732899463615,
 5.4272142464351605,
 -0.24666106139435517,
 5.240692361413373,
 2.2080702184753753,
 5.634467849972537,
 3.0192102947549166,
 2.0654794049088796,
 12.328104960830139,
 3.851848968425272,
 6.420571828775482,
 2.1735102786835894,
 6.6844604659852,
 -2.054535641573677,
 3.5238853331948485,
 1.2976642249404085,
 2.558838697293604,
 -18.85072264468609,
 9.388685521210833,
 8.064133661081565,
 3.6545421862854246,
 -4.4874759130284225,
 1.7749049238218,
 0.6305713284282508,
 2.1509827723852872,
 13.836588931578351,
 11.801669837755716,
 16.868761182116994,
 8.377425544249212,
 0.8482497544992285,
 8.118837972350118,
 16.096582337725742,
 12.20144687728482,
 0.8359898032729518,
 6.125916131559685,
 8.290768161718413,
 10.256793764569341,
 3.8400356830579168,
 2.873058717541006,
 14.826628578938063,
 2.334186121946921,
 -0.07041852363003515,
 0.5466731470273676,
 2.9458202497799215,
 8.871524805754339,
 5.194137487156157,
 18.354568602432945,
 3.4189392385557675,
 14.407958564847505,
 6.955843199059968,
 6.484400335725655,
 15.09729761536745,
 4.013705405379098,
 10.438470261576647,
 9.343928098617903,
 8.954152299745656,
 9.924457183557726,
 5.169621995981508,
 23.18221743036945,
 11.31906098501505,
 8.096673778893514,
 1.3728520667891995,
 18.098389146912105,
 -3.1350703681532255,
 4.8613570598631135,
 3.856761635517545,
 3.759613550004225,
 5.364450007095185,
 0.5783995797338382,
 9.342763361467862,
 16.584058350147092,
 1.9920301090263055,
 3.7674029757561973,
 5.971388540265602,
 2.6311393224855237,
 15.515466734016469,
 6.647444203767158,
 8.246162759557674,
 8.468996115766434,
 10.80144040674628,
 2.838621437462591,
 9.562422692663175,
 4.621850524646787,
 14.684917837767902,
 16.565923903845587,
 9.508312050937315,
 16.442310734809425,
 -0.8839001379119905,
 7.572799303080511,
 6.343617168441694,
 8.033392705323283,
 14.1609728022387,
 7.859492009249407,
 8.267961978478368,
 15.893894086580973,
 18.859144355048596,
 5.73646107189737,
 1.079670778747571,
 -3.695435594648914,
 4.13656410216965,
 2.3150333989157503,
 23.68394474330985,
 3.562114104376942,
 4.450397947696039,
 6.875209811110077,
 2.4987814423186725,
 9.941482544796791,
 1.7925872305835089,
 19.513558433685088,
 6.460758559000458,
 3.349784204192775,
 1.9740300719609658,
 18.16836915651938,
 3.110959702174036,
 0.11243565066176502,
 17.15333238883396,
 15.21512356715115,
 3.125996851354263,
 2.962068590497029,
 -3.5624172423090124,
 13.135713766887623,
 -1.3301433383211068,
 7.373543532413682,
 12.298574774234481,
 -9.54863336115516,
 15.356686374970668,
 8.603953348034425,
 7.2615895791579534,
 12.867834937616971,
 1.1316814982172745,
 11.814418874101197,
 2.1173256905728626,
 0.11206240379793417,
 3.564188717658537,
 18.475291798443326,
 12.026156262218477,
 8.576181352832766,
 18.094279649528392,
 0.6679691460216457,
 11.787456777849298,
 11.069045550729044,
 10.795242379080888,
 2.4865814481622746,
 8.071072151326861,
 -0.21979172584496004,
 3.507098205501669,
 0.4655312477753114,
 -1.4389178470839443,
 4.370241039500666,
 5.250475159120487,
 1.8352124998710695,
 -2.0918506149762695,
 -4.531636381835662,
 0.8997162292626912,
 16.267572472406126,
 16.2868185144463,
 1.2433738125210834,
 -13.166083709483354,
 -17.124820805599363,
 0.9359412378416824,
 11.205495515888616,
 7.061396423077718,
 5.612671537675956,
 5.61396148494895,
 6.799492039287728,
 17.61150032010714,
 12.690223748443305,
 0.6298230824498017,
 0.729948628028282,
 1.0326463812880484,
 0.5907410591294384,
 7.183956314992773,
 11.70478971719716,
 1.9265363753558373,
 0.691248979072786,
 6.0863171389360025,
 0.9834053375649043,
 4.936429651628489,
 4.337130453055484,
 -4.574015321841423,
 4.593681532370055,
 -0.346597265369764,
 14.136530458644096,
 2.6838214942953718,
 -6.191077624187653,
 8.114083547493527,
 2.8198091623537853,
 11.213974849070945,
 1.7787653908040628,
 -11.13372497307946,
 4.964868853373067,
 5.782460849614798,
 3.91795561066937,
 1.0898894019285659,
 5.683019416767933,
 3.828981462562596,
 2.544006080764948,
 2.816168046658628,
 4.261414278285487,
 0.924390827213962,
 7.495168251936568,
 14.042457730829573,
 11.875403726996105,
 9.101492482907194,
 4.412643731505019,
 13.767541117114897,
 1.5060423713191458,
 1.3969382302176845,
 -0.937460244930717,
 4.102969080605341,
 12.869574058800769,
 3.3713547931282255,
 3.5933583809055856,
 19.540225697949694,
 1.3363261815218224,
 3.6831232806943834,
 5.15382878417063,
 -0.9551578559957241,
 -5.335899358755328,
 13.907071401829633,
 2.713363149906829,
 -0.6436727729611025,
 2.8458900775031664,
 12.014882696656622,
 13.168550846645147,
 4.7121715541817375,
 0.007821818858896566,
 1.8369609623906322,
 14.893220823152676,
 13.596220921366127,
 18.700840054758718,
 2.568598941405596,
 9.079917760696446,
 -1.0984560231443041,
 1.6632740511612256,
 4.127615166358282,
 12.063218786062993,
 13.64822474505647,
 4.242538560373163,
 1.2003512267301597,
 9.261564050144209,
 -2.850738445740878,
 6.031150778729164,
 -0.8548399553944112,
 6.961419673138146,
 13.68218368617604,
 22.613927094527007,
 1.4597486153997614,
 10.6054034361956,
 12.790851573936283,
 -9.230892750581475,
 13.693671110381068,
 0.13236857885695522,
 6.093063805410051,
 2.287326027338122,
 -0.3370132809766107,
 7.16133727427061,
 0.9859291819352305,
 10.324690733020043,
 7.008814607122613,
 2.56099186863215,
 6.313040938750758,
 11.758777622149745,
 0.7081331711366451,
 6.5944591315563,
 9.532971204277771,
 11.383535827377832,
 9.998479233979701,
 1.6229755044080738,
 15.446354890839286,
 14.90472584724668,
 11.525123275957318,
 3.084477347120773,
 -11.45064212936713,
 -4.7710543507637055,
 14.48455362257623,
 0.873635668096832,
 4.306902993635566,
 6.7928709769708115,
 6.037792360330172,
 8.08200813090284,
 9.351359544757877,
 8.052855770201141,
 -3.164857603557859,
 8.414820209348438,
 9.48406182524283,
 6.941184425567794,
 14.965050151546668,
 12.079552177848306,
 2.6702563541733615,
 20.766962322327448,
 1.4034200564504162,
 12.626476188842693,
 6.931062597433528,
 4.843938342175694,
 4.012000435181328,
 2.2279831163907478,
 -1.2904058370410951,
 0.6332649095151476,
 8.894389994527511,
 2.519537825568831,
 15.245282878296123,
 -3.742507144322874,
 3.591886935522443,
 18.92776007651339,
 -19.44189371116791,
 -10.637473889487305,
 -7.200381914050443,
 4.099105080743405,
 -2.320016013204828,
 2.340875982956661,
 -0.7746195098696376,
 3.0060422352474774,
 11.078665247589198,
 6.070613226766904,
 3.526921777970891,
 17.016002869016813,
 12.697991122903742,
 2.6616695837018516,
 12.55415225605964,
 13.604417635837219,
 12.706252426051826,
 12.265846069516796,
 5.763370871894978,
 -0.1203691104257681,
 -2.8096919542358982,
 6.393453751431049,
 -0.7257492892918262,
 -2.618519735394932,
 6.48274802141853,
 11.826656892677308,
 0.5709950914385115,
 6.851535802302768,
 8.708852890362447,
 -16.72719573868665,
 -6.53287530619968,
 -1.8382433715515758,
 1.349399517063492,
 5.376698685392138,
 0.475227014388839,
 11.915352036253694,
 8.565767480920996,
 3.1509670055122077,
 8.364286610898235,
 0.20067117308099133,
 6.287120403738762,
 -0.6417372564493623,
 5.325038203966845,
 13.184060942373458,
 4.431163766216633,
 5.134673016978456,
 10.56717079171028,
 8.854685390863656,
 8.89267063059854,
 5.458147909306203,
 2.571854036074405,
 10.258598407395434,
 11.314082358396893,
 5.546034284514716,
 5.903027267804637,
 -0.2864711687446488,
 3.9375156559017657,
 0.8353063612055782,
 9.29658187255185,
 12.647496749352905,
 -2.5855640050057294,
 -6.614431882649772,
 -1.2749557918529506,
 -41.857603630758526,
 -12.388381041619844,
 -0.2880608540522256,
 -7.799301448679392,
 -19.84218172398691,
 -14.779296608038996,
 -14.549015776685563,
 -4.4567970647886925,
 -16.48511881063415,
 -16.575384169560877,
 -32.115802074402986,
 -3.0636570480005534,
 -6.7825218807386385,
 -0.168014592129856,
 -4.021484305083434,
 -0.5856619194644337,
 -0.9478425170660985,
 -1.771623948156478,
 1.982221165044443,
 -4.874510298445607,
 -3.5461014635202277,
 -10.30876277629429,
 -9.798523278888489,
 -3.464097419915775,
 -21.80519102092397,
 -23.55314014067687,
 -5.896894349416295,
 -0.3759661688583035,
 9.32807625860695,
 -15.62230659132006,
 -7.498252980816859,
 2.7660376177166346,
 -3.0040804706631907,
 -28.66466994170639,
 -19.956984117158107,
 -22.481912041985208,
 -19.148359369619623,
 0.3763199140920464,
 0.06806480808642096,
 -3.1556538529642384,
 -9.840101415809803,
 -21.138224782769253,
 -16.184055634666358,
 -12.843415417201774,
 -2.800150799604174,
 -6.974984931787916,
 2.293179873395375,
 -21.088606849363224,
 -1.9227237015451437,
 -3.2009359465777507,
 1.0769926645501717,
 -15.214282732463467,
 -6.451331706188897,
 -25.97366691491358,
 -2.0874712806285247,
 -1.6114507496564996,
 -21.602960399148312,
 -24.528004599526078,
 -5.828775404221951,
 -22.992245988725344,
 -24.27226967053495,
 -16.862369297474007,
 0.26019343186616767,
 0.49344065089837424,
 -0.3371168162088544,
 -0.8950327162301122,
 -3.3201796416339233,
 -13.523930143870482,
 -4.799804929140886,
 -3.5426895954764364,
 -7.601339075572869,
 -5.479085455650956,
 -0.7001741588220123,
 -5.226541622172647,
 -14.988747253000948,
 -1.9008644499022105,
 -22.11433726831844,
 1.4999546239323436,
 -13.181346218245928,
 -4.777538175249851,
 -2.8559086145580976,
 -6.066868827406821,
 -6.423157467850443,
 -1.1501510103450086,
 -4.453748477462199,
 -20.132801583400173,
 -0.7274922931289804,
 -1.5117470282173382,
 -3.343078401833221,
 -5.531493606332967,
 -13.776427749971267,
 -7.584994823887012,
 -5.941466427033823,
 -12.05113723185431,
 -10.73736105943782,
 -10.344375022247512,
 -0.0381819082942122,
 -5.2589934682006,
 -5.137522854300959,
 -9.17116017534915,
 -21.541281101821287,
 -22.492181745834046,
 -8.779664494211056,
 -5.2211236291820535,
 1.143431487978797,
 0.1013100997432261,
 -6.9558061684421055,
 -8.627827929044985,
 -4.4392667137350745,
 0.8874268503438125,
 -1.3389566057204108,
 -11.116440362214927,
 -22.489822707411168,
 -11.866747472644743,
 -5.1429097754745765,
 -7.394001013314859,
 -4.252944635747917,
 -3.279542940311437,
 -0.141986066584408,
 1.4670545865453462,
 -5.80036304305845,
 2.125439315389172,
 3.0909950256902485,
 -6.013773344241271,
 3.242015573025861,
 -6.093883371047832,
 -9.699076282878481,
 -5.000476500261804,
 -6.170737427136147,
 -6.105021739635699,
 -3.513197797946055,
 -14.103583095091079,
 -13.050478922576062,
 -7.40757789543704,
 -13.05711925527225,
 -7.327511398291972,
 -14.211743384961979,
 -9.466594695453143,
 1.0166012420607757,
 -3.5030914814396525,
 -27.93987382215348,
 -12.936976265362679,
 -19.176271046040394,
 -22.74543367433306,
 -23.646615526919067,
 -1.9860057157635498,
 -6.153200915744493,
 1.806972764245981,
 -11.988615694733662,
 -6.6881227347379815,
 -2.1757190721233606,
 -1.0055543372563704,
 -0.13011736133198887,
 -7.142809804634096,
 -6.462463656834884,
 -10.51294502117066,
 -0.3064569412622204,
 -20.751542759357804,
 -0.4009171259937992,
 0.9359443522553794,
 -16.186599141524862,
 -1.7213137395502418,
 -7.312247567794082,
 1.6996304967583267,
 -13.553993640137543,
 2.621117312234707,
 -1.382504401691234,
 -14.872859135951455,
 -1.7532805953778405,
 -2.5701501931638218,
 -1.9064149302082032,
 -12.516654077471488,
 -10.898212664432647,
 -13.399716232411759,
 -1.8383239098510795,
 -0.5301098893253893,
 -4.659018255002846,
 -5.065011710602443,
 -0.44510269064364155,
 -7.124696470762078,
 -9.919669079147969,
 2.9532870855398774,
 -14.952262854683124,
 -6.920668637485053,
 -11.844408048291944,
 0.5218821665220238,
 -6.002809730652897,
 -3.234987593183222,
 -15.465415188335419,
 -6.5728224179335335,
 -2.416106329666751,
 -9.81929371353461,
 0.3167378045863061,
 -19.76691605257122,
 -18.893769983171133,
 -11.335585316903956,
 -2.3616528546154667,
 -22.944948869957,
 0.5438998583780783,
 -9.596855981247302,
 -14.252037035543644,
 -10.961316404495907,
 -13.186587882935743,
 -4.960721930828123,
 -2.0259752093147188,
 -0.8126619899797926,
 -0.38980727995093617,
 -14.538182021050595,
 -7.9232030366717385,
 -17.92516514913903,
 -19.430528569845944,
 -1.0961681069979063,
 -12.02163856321367,
 -14.525210551967799,
 0.2163687599640079,
 -7.017492770802578,
 -0.07040760390230061,
 -22.26787869185262,
 -1.7322407114930958,
 -12.850630704759137,
 -16.257187013412505,
 0.22756910804826141,
 -0.21232257015338116,
 0.09833637720113941,
 -20.440580507219583,
 -9.71538157546101,
 -0.7666942610766433,
 -12.419041065072767,
 -3.9613718022501105,
 3.476609787115222,
 -15.422679699107208,
 -1.089879316917509,
 0.5000033464144099,
 2.7312334126271347,
 -3.119812493373942,
 -1.9493929337748883,
 -10.691882099994421,
 -14.34620298956947,
 -12.268122458081496,
 -14.66212198478227,
 -6.6785001482776,
 -2.958150327843708,
 -14.966364589640374,
 0.9005829533960681,
 -15.156196752856866,
 -5.455682164576218,
 -9.74601498284705,
 -11.990268054586345,
 -1.1550259555996194,
 -12.748434408812674,
 -4.1135805999795885,
 -1.6521575474970929,
 -4.182283350239005,
 1.477309029451755,
 0.7032862876764551,
 -5.531485596621507,
 -18.03212759789605,
 -8.03203027617602,
 3.7316812234979717,
 -3.16885223949476,
 -0.001781566871595595,
 -6.942630067550773,
 -11.459944702793344,
 -1.0923419225134718,
 -1.2344310135868741,
 -2.06661241017753,
 -0.16795252238842778,
 -5.837442074638025,
 -6.969048609428475,
 -6.436517926967075,
 0.7855520900166658,
 -11.486530955932341,
 -2.4324279700490523,
 -19.453884545979896,
 -5.766730836693703,
 -4.982175222421134,
 -24.40811118581756,
 -10.949777968489132,
 -6.8113863716179575,
 -22.876022314288793,
 -5.933293381923468,
 -5.779417228814326,
 -8.682723192190307,
 -5.177817699661105,
 -20.161578622650183,
 -7.380423949911887,
 -9.20710785906314,
 -2.2674624955626275,
 -1.3006304049996813,
 -2.1908887276834723,
 -0.8623290611628646,
 -13.24501980333525,
 -0.7047255095462276,
 5.127393548282441,
 -6.746072177763425,
 -6.4149742930254545,
 -12.907720277839932,
 -6.785919649192354,
 3.0021360636277574,
 -1.5665499275644421,
 -15.213379232779124,
 -13.781627008097464,
 -5.955686475314103,
 -7.717230505725417,
 -2.885965144007942,
 -0.7774343839804209,
 -0.5090807180077122,
 -8.49688882353506,
 -8.994635087626584,
 -13.286819811543664,
 -4.879677170493555,
 -1.4678151205743009,
 -8.44430490645,
 -3.529987103147294,
 -10.688146337814668,
 0.1999290258144286,
 -6.980205397962368,
 -11.509303158541366,
 -11.573891892065285,
 -0.5845892672463363,
 6.945183847103087,
 -15.556365809447472,
 -19.13703353073335,
 -1.84780555915496,
 -1.4178283233864166,
 -2.806454854883191,
 -3.0168900157903136,
 -0.7065968434763532,
 -11.372410437303275,
 -15.106536851575626,
 -9.243183080170443,
 0.37055790722689097,
 -5.071875822063377,
 -5.285252132425524,
 -1.102319826494791,
 1.2497968362361256,
 -0.7087276585774384,
 -8.83773554829272,
 -4.8030190056834785,
 -14.980821722889921,
 -13.716130852354162,
 -11.846707909615679,
 0.623705445903368,
 -2.3992612034055907,
 -7.047320876805297,
 -12.896831133844355,
 -5.351376434855872,
 -0.2553982834317934,
 -0.759774937685913,
 -1.731404267427618,
 -3.4392678461932076,
 -5.278552716074528,
 -9.296606570182448,
 -0.9997778255968341,
 2.8783599245496383,
 -7.3214284306533,
 -14.65903909129867,
 0.4948173763671235,
 -15.837474875818527,
 -0.25586931910818866,
 -14.050530141610292,
 -14.496213783717684,
 -8.307341703948795,
 -6.185320730239342,
 -14.282760984531937,
 -0.8910055410176208,
 1.834310510527041,
 4.670379372792209,
 -2.9963567143400645,
 -13.81157654949501,
 -1.448959748674119,
 -2.406155701837233,
 -1.6922332264767637,
 -7.7988621938049665,
 -5.628325707671162,
 -10.78756734188156,
 -2.84424892812351,
 -16.106860799658524,
 -0.2791805294114049,
 -1.4764534398231868,
 -18.51714312470754,
 -24.821715787173627,
 0.08954469475725789,
 -7.057848405897688,
 -1.2577921354278518,
 0.9149403372099462,
 -5.5736638364710425,
 -4.501275017594805,
 0.429748697388043,
 -3.003757748909888,
 -12.446769595077601,
 -2.47542254642112,
 -1.9202484624115037,
 -1.371114087387897,
 -2.717787543074314,
 -3.0577474253861165,
 -12.996162899508173,
 -15.762017490854149,
 -1.1790989583740998,
 -4.401856665591766,
 -10.047536342425667,
 -9.836288818651447,
 -9.918844787664971,
 -0.10010301230260334,
 -2.588273703999478,
 2.990132601663112,
 -17.569224671460304,
 0.24541609912511106,
 0.1239108185818368,
 8.16325074918232,
 -6.97714741810066,
 -17.471106592389162,
 -9.971941446187666,
 -11.274665176218681,
 -6.800512275062238,
 0.3092515647498086,
 -9.564443123517496,
 0.49658896082011816,
 -5.535432222472048,
 -0.6067091690407102,
 -5.987248581814984,
 -6.35455860668333,
 -7.316674370430985,
 -5.755616553476967,
 -1.450606606633504,
 -21.1132790554416,
 -5.872285031515529,
 -0.6465023363659449,
 2.15866894236554,
 -2.4704641293720013,
 -11.881160781063423,
 -11.133598190934588,
 -5.046757270314156,
 -4.5356152571557375,
 -2.8186465888024035,
 -10.28062784270197,
 4.516987614876636,
 -1.3968432219532194,
 -2.122563615308011,
 -9.859009411427335,
 1.7285394987754827,
 -6.365619547394054,
 -2.0215328266816406,
 -8.748969405690573,
 -9.352501757802948,
 2.3168870293364106,
 -17.047320786260812,
 -1.1773167328150358,
 -9.707591811716107,
 -14.28188017943252,
 -0.07366509157359745,
 4.92892814621812,
 -12.909824420342575,
 4.891465666693185,
 2.419314038829235,
 2.5374115498496828,
 -5.7761799924336925,
 -7.268489096988577,
 -15.538238970752776,
 5.705728079906604,
 -9.033309703787653,
 -10.719797332755867,
 -3.354717023447467,
 -0.596389751177137,
 -14.500799323910037,
 -9.68030434459526,
 5.850004999829273,
 -8.386858654393837,
 -1.3823590708845062,
 -5.531159001913358,
 2.047545808165412,
 1.94674758543843,
 -5.06107651632409,
 -17.646261397465928,
 -16.38957307442553,
 -3.58331235105713,
 -6.20622203931851,
 -8.662284956092664,
 3.2412690614744064,
 -3.326005237280391,
 -5.153086273248228,
 -2.0530479298274527,
 -0.15007298578673137,
 -17.06607920469751,
 -6.141732028375991,
 -9.387684143530361,
 -14.603501977551685,
 -2.077438150139261,
 11.20847944973689,
 -4.382633292747986,
 -13.707013788733788,
 -4.6563983908907485,
 -5.338366219219153,
 -0.8323813177592969,
 1.3187080887647809,
 5.2614896460457645,
 1.3255159139908588,
 -3.704700076391068]

In [22]:
test_data = [[word_tokenize(dd[1]), 0] if dd[0]=='0'  else [word_tokenize(dd[1]), 1]  for dd in data[799000:801000]]

In [21]:
for f in test_data:
    print f[1]

In [23]:
error = 0
for d in test_data:
    x = twitte_to_x(d[0], key_words)
    wx = t.list_contrac(x, w)
    p = logist(wx)
    print d[1]
    if (p-0.5)*(d[1]-0.5)<=0:
        error +=1
print error

In [ ]:
# error rate is about 30%